# SD Card Decoding

## Protocol Outline


![image](https://i.imgur.com/OQpyljX.png)  
  
Reference: [Physical Layer Simplified Specification](https://www.sdcard.org/downloads/pls/) 

### Protocol Details (thks chatGPT for summarising)
1. **Command Frame (CMD24, WRITE\_BLOCK)**: The host sends a command frame to the SD card to initiate a write operation. This command frame is usually CMD24 (in binary: `01011000`, in hexadecimal: `0x58`). The structure of the command frame is:

   - Start Bit (1 bit): Always `0`.
   - Transmission Bit (1 bit): `1` for host-to-card.
   - Command Index (6 bits): `011000` for CMD24.
   - Argument (32 bits): Address of the block to be written. Given as a 32-bit binary number.
   - CRC (7 bits): A CRC-7 checksum of the command frame, used for error detection.
   - End Bit (1 bit): Always `1`.  
   
2. **Response Frame (R1)**: The SD card responds with an R1 response frame, which includes a status code indicating whether the command was accepted or if there were any errors. The structure of the R1 response frame is:

   - Start Bit (1 bit): Always `0`.
   - Transmission Bit (1 bit): `0` for card-to-host.
   - Reserved Bits (2 bits): Reserved for future use, usually `00`.
   - Command Response Token (4 bits): Indicates the status of the SD card.
   - CRC (7 bits): CRC-7 checksum.
   - End Bit (1 bit): Always `1`.

   The R1 response is a byte in length. If the SD card accepted the command, the response will be `0x00`. If there were errors, other values might be returned, such as `0x01` for an idle state or `0x02` for an erase reset error, among others. These values are usually the bitwise OR of different error codes. The response is sent out least significant bit (LSB) first.

3. **Data Token (aka Block Token)**: After receiving a valid response, the host sends a data token to signal the beginning of the data block. The data token for a single block write is `11111110` (in hexadecimal: `0xFE`).

4. **Data Block**: The host then sends the data block, which contains the actual data to be written to the SD card. The size of the data block is usually 512 bytes, though this can vary depending on the SD card's block size. This is done most significant bit (MSB) first.

5. **CRC**: After the data block, the host sends a 16-bit CRC (CRC-16) for error detection. The SD card uses this CRC to check the integrity of the data block. However, in SPI mode, the SD card does not check this CRC value unless the host has previously sent the CMD59 command to enable CRC checking. Many libraries, including the Arduino SD library, do not use this command and the CRC check is typically disabled for simplicity. The CRC sent by such libraries is often a fixed, dummy value.

6. **Data Response**: The SD card responds with a data response token, which indicates whether the data was accepted, if there was an error in the data (e.g. a CRC error), or if the SD card is busy. The structure of the data response token is:

   - Token Value (3 bits): `010` means data accepted, `101` means CRC error, `110` means write error.
   - Reserved (1 bit): Reserved for future use, usually `0`.
   - End Bits (4 bits): Always `1111`.

   - If the data is accepted, it responds with `0x05` (`XXX00101` in binary, where `X` can be any value). This indicates that the data block was accepted and written successfully.
   - If the SD card detects a CRC error, it responds with `0x0B` (`XXX01011` in binary). This means that the data block was not written due to a CRC error.
   - If there was a write error, the card responds with `0x0D` (`XXX01101` in binary). This means the data block was not written due to an internal error in the card.
   - The Data Response is sent out LSB first. The host must check the data response to verify that the data block was written correctly. If there was an error, the host may choose to retry the write operation.




## Challenge

### Arduino Src Code


```c
#include <SPI.h>
#include <SD.h>

bool toExecute = true;

void setup()
{
    // Init Serial
    Serial.begin(115200);

    // Test and Init SD Card
    Serial.println("Test SD Card Init");
    if (!SD.begin(4))
    {
        Serial.println("FAILED");
        while (1);
    }
    Serial.println("OK");
}

void loop()
{
    // wait for serial input before executing following code. once executed, never execute again
    while (!Serial.available())
        ;
    String input = Serial.readString();
    if (input != "start")
    {
        Serial.println("Invalid input");
        return;
    }
    else if (input == "start" && toExecute)
    {
        Serial.println("Starting");
        // Create flag.txt file if not already created
        if (SD.exists("flag"))
        {
            Serial.println("flag already exists");
            return;
        }
        else
        {
            Serial.println("Creating flag file");
            File flag = SD.open("flag", FILE_WRITE);
            flag.println("ACSI{d1d_y0u_r34d_th3_SD_SPI_sp3c5?}");
            flag.close();
        }
        toExecute = false;
        Serial.println("Done");
    }
}
```

### Solution

In [15]:
import csv

def decode_sd_card(file_name):
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header

        data_blocks = []
        current_block = []
        start_write = False
        got_r1 = False
        have_data_block = False

        for row in reader:
            event_type = row[1]  # assuming type is in the second column
            mosi = row[4][2:] if row[4] else ''  # assuming MOSI data is in the fifth column, remove '0x'
            miso = row[5][2:] if row[5] else ''  # assuming MISO data is in the sixth column, remove '0x'

            # Only process 'result' events
            if event_type != 'result':
                continue

            # Check if we've found the CMD24 command (0x58)
            if mosi == '58':
                start_write = True
                continue

            # After CMD24, SD card should respond with 0x00
            if start_write and miso == '00':
                start_write = False
                got_r1 = True
                continue

            #After getting R1 response, wait till start of data block (i.e data token)
            if got_r1 and mosi == 'FE':
                got_r1 = False
                have_data_block = True
                continue

            # Start collecting data after data token (0xFE)
            if have_data_block:
                current_block.append(mosi)

            # Stop collecting data after 512 bytes or when we receive data response (0x05) from SD card
            if len(current_block) == 512 or miso == '05':
                data_blocks.append(current_block)
                current_block = []
                have_data_block = False

    # Convert blocks to ASCII and print
    for block in data_blocks:
        print(''.join(chr(int(b, 16)) for b in block))

In [16]:
decode_sd_card('txt_file_spi.csv')

FLAG    TT   !(!(  !(    FLAG          !(!(  !(                                                                                                                                                                                                                                                                                                                                                                                                                                                                      ÿ

øÿÿÿÿÿÿÿ                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      